# SUMMARY

まずはじめに運営/参加者の皆様ありがとうございました。

野球は好きなスポーツなのでデータを見て楽しみながら参戦することができました。
Oreginさんが圧倒的なスコアで序盤から最後まで1位で走り切ったのが印象的なコンペになりました。
個人的な目線では、この予測ターゲットはかなり運ゲーなのではないかと最後まで思っていました。だからこそOreginさんのスコアは衝撃的でした。

1位とは大きく離されたスコアでアプローチも非常にシンプルなものでおそらくみなさんにとってあまり驚きのあるものではないと思いますが、供養のために解法をシェアします。

[アプローチ]

1.　前処理

今回のコンペにおいて、前処理は非常に重要なものだったと思います。 

特に[DT-SNさんのシェア](https://prob.space/competitions/npb/discussions/DT-SN-Post2126e8f25865e24a1cc4)で使われていた出塁状態、ボールストライクカウントの数値化はかなり有効でした。　

当初はカテゴリ変数として扱っておりスコアが出なかったのですが、この数値化により大きくスコアを改善しました。これは勉強になりました。　

また打者投手の利き手などに欠損が見られた部分の補完は、両打ち実績がある打者の場合は投手の利き手とは逆の手を補完する判定を組み込むなど、なるべくデータを綺麗にすることを努めました。　


2. 特徴量エンジニアリング

基本的には集計特徴量をベースにしています。

また試合ごとの打者の出現順番の特徴量（厳密には打順ではないですが、ここでは打順と呼びます。）も効きました。
これに通じる試合における打者/投手/打順のTfidfも効きました。 
興味深かったのが、打者（batterCommon）ごとの打順（batting_order_in_subgameID）の統計特徴量がテストスコアに対しては有効でした。

3. モデル

LightGBMの5seeds平均アンサンブルです。
個人的なポイントは、今回のタスクではローカルCVスコアを上げすぎるとパブリックスコアが大きく下がってしまう傾向がありました。
そのため過学習せぬようmax_binパラメータを小さくするなどの工夫を行いました。

4. Fold分割

gameIDごとのRandomGroupKFoldです。当初StritifiedGroupKFoldも試していましたがリーダーボードとの相関が高かったのは結果RandomGroupKFoldでした。


以下解法のnotebookです。また対戦宜しくお願い致します。

In [1]:
# ----- Import common library -----
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from tqdm import tqdm_notebook as tqdm
from glob import glob
import gc
import pickle
from time import time, sleep
import json
import pytz
import random
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display

In [2]:
INPUT_DIR = './input'

In [3]:
# ====================================================
# CONFIG
# ====================================================

class CONFIG():
    def __init__(self):
        self.debug = False
        self.target = 'y'
        self.num_class = 8
        self.sampling_num = 10
        self.seeds = [2021, 2022, 2023, 2024, 2025]
        self.how_split = 'RandomGroupKFold'
        self.n_splits = 5
        self.group_col = 'gameID'
        
CFG = CONFIG()
    
print (f"{CFG.__dict__}")

{'debug': False, 'target': 'y', 'num_class': 8, 'sampling_num': 10, 'seeds': [2021, 2022, 2023, 2024, 2025], 'how_split': 'RandomGroupKFold', 'n_splits': 5, 'group_col': 'gameID'}


In [4]:
def read_data(input_dir):
    # 投球結果(0:ボール, 1:ストライク, 2:ファウル, 3:アウト, 4:シングルヒット, 5:二塁打, 6:三塁打, 7:ホームラン)
    train = pd.read_csv(f'{input_dir}/train_data.csv')
    test = pd.read_csv(f'{input_dir}/test_data_improvement.csv')
    game_info = pd.read_csv(f'{input_dir}/game_info.csv')
    print(f'train shape = {train.shape}')
    print(f'test shape = {test.shape}')
    
    sample_submission = test[['id']].copy()
    sample_submission['y'] = 0
    print(f'sample_submission shape = {sample_submission.shape}')

    train['test'] = 0
    test['test'] = 1
    df = pd.concat([train, test]).reset_index(drop=True)
    df = df.merge(game_info, on=['gameID'], how='left')

    df.drop(columns=['Unnamed: 0'], inplace=True)
    df = df.drop_duplicates(['totalPitchingCount', 'B', 'S', 'O', 'pitcher', 'batter', 'gameID', 'inning', 'startDayTime'])
    df['startDayTime'] = pd.to_datetime(df['startDayTime'])
    df['date'] = df['startDayTime'].dt.date
    df = df.sort_values(['startDayTime', 'gameID', 'inning', 'O', 'totalPitchingCount']).reset_index(drop=True)
    return df, sample_submission

def create_diffence_team_feature(topTeam_values, bottomTeam_values, inning_top_values):
    new_values = topTeam_values.copy()
    new_values[inning_top_values==0] = topTeam_values[inning_top_values==0].astype(object)
    new_values[inning_top_values==1] = bottomTeam_values[inning_top_values==1].astype(object)
    return new_values

def create_offence_team_feature(topTeam_values, bottomTeam_values, inning_top_values):
    new_values = topTeam_values.copy()
    new_values[inning_top_values==1] = topTeam_values[inning_top_values==1].astype(object)
    new_values[inning_top_values==0] = bottomTeam_values[inning_top_values==0].astype(object)
    return new_values

def create_pitcher_team_feature(pitcher_values, topTeam_values, bottomTeam_values, inning_top_values):
    new_values = pitcher_values.copy()
    str_values = np.full(new_values.shape[0],"@")
    new_values[inning_top_values==0] = pitcher_values[inning_top_values==0].astype(str).astype(object) + str_values[inning_top_values==0] + topTeam_values[inning_top_values==0].astype(object)
    new_values[inning_top_values==1] = pitcher_values[inning_top_values==1].astype(str).astype(object) + str_values[inning_top_values==1] + bottomTeam_values[inning_top_values==1].astype(object)
    return new_values

def create_batter_team_feature(batter_values, topTeam_values, bottomTeam_values, inning_top_values):
    new_values = batter_values.copy()
    str_values = np.full(new_values.shape[0],"@")
    new_values[inning_top_values==1] = batter_values[inning_top_values==1].astype(str).astype(object) + str_values[inning_top_values==1] + topTeam_values[inning_top_values==1].astype(object)
    new_values[inning_top_values==0] = batter_values[inning_top_values==0].astype(str).astype(object) + str_values[inning_top_values==0] + bottomTeam_values[inning_top_values==0].astype(object)
    return new_values

def fillna_pitcherHand(df):
    pitcherHand_df = df[pd.notnull(df['pitcherHand'])].groupby('pitcher')['pitcherHand'].max().reset_index()
    df.drop(columns=['pitcherHand'], inplace=True)
    df = df.merge(pitcherHand_df, on='pitcher', how='left')
    return df

def batter_isPitcher(df):
    pitcher_df = df[pd.notnull(df['pitcherHand'])].groupby('pitcher').size().reset_index()
    pitcher_df['batter'] = pitcher_df['pitcher']
    pitcher_df['batter_isPitcher'] = 1
    pitcher_df = pitcher_df[['batter', 'batter_isPitcher']]
    df = df.merge(pitcher_df, on='batter', how='left')
    df['batter_isPitcher'] = df['batter_isPitcher'].fillna(0)
    return df

def convert_batterHand(x, batterHand_dict):
    try: 
        return batterHand_dict[x]
    except: 
        return pd.np.nan

def fillna_batterHand(df):
    batterHand_nunique = df[pd.notnull(df['batterHand'])].groupby('batter')['batterHand'].nunique()
    doubleHand_batter = list(batterHand_nunique[batterHand_nunique==2].index)
    cond = (pd.isnull(df['batterHand'])&(df['batter'].isin(doubleHand_batter)))
    df.loc[cond, 'batterHand'] = df.loc[cond, 'pitcherHand'].map(lambda x: {'R': 'L', 'L': 'R'}[x])

    batterHand_dict = df[pd.notnull(df['batterHand'])].groupby('batter')['batterHand'].max().reset_index().to_dict()
    cond = pd.isnull(df['batterHand'])
    df.loc[cond, 'batterHand'] = df.loc[cond, 'batter'].map(lambda x: convert_batterHand(x, batterHand_dict))

    cond = pd.isnull(df['batterHand'])
    df.loc[cond, 'batterHand'] = df.loc[cond, 'pitcherHand'].map(lambda x: {'R': 'L', 'L': 'R'}[x])
    return df

def create_base_features(df):
    
    df['BS'] = df['B']*(10**0) + df['S']*(10**1)
    df['BSO'] = df['B']*(10**0) + df['S']*(10**1) + df['O']*(10**2)

    df['inning_num'] = df['inning'].map(lambda x: float(x.split('回')[0]))
    df['inning_num'] = df['inning_num'] * 2
    df['inning_top'] = df['inning'].map(lambda x: 1 if x.split('回')[-1]=='表' else 0)
    df['inning_num'] = df[['inning_num', 'inning_top']].apply(lambda x: x['inning_num']-1 if x['inning_top']==1 else x['inning_num'], axis=1)
    df['inning_num_half'] = df['inning_num'] // 2
    df['out_cumsum'] = (df['inning_num_half']-1)*3 + df['O']
    
    place_dict = {
        'PayPayドーム': 0, 
        '京セラD大阪': 1, 
        'メットライフ': 2,
        '横浜': 3, 
        '神宮': 4, 
        '東京ドーム': 5, 
        'ZOZOマリン': 6,
        '楽天生命パーク': 7, 
        'ナゴヤドーム': 8, 
        '札幌ドーム': 9, 
        'マツダスタジアム': 10, 
        '甲子園': 11, 
        'ほっと神戸': 12
    }
    df['place'] = df['place'].map(lambda x: place_dict[x])

    df['pitcherTeam'] = create_diffence_team_feature(df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)
    df['batterTeam'] = create_offence_team_feature(df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)
    df['pitcher'] = create_pitcher_team_feature(df['pitcher'].values, df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)
    df['batter'] = create_batter_team_feature(df['batter'].values, df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)

    # trainとtestに共通のピッチャーを取得
    train_pitcher = set(df[df['test']==0]['pitcher'].unique())
    test_pitcher = set(df[df['test']==1]['pitcher'].unique())

    # trainとtestに共通のバッターを取得
    train_batter = set(df[df['test']==0]['batter'].unique())
    test_batter = set(df[df['test']==1]['batter'].unique())

    df['pitcherCommon'] = df['pitcher']
    df['batterCommon'] = df['batter']
    df.loc[~(df['pitcherCommon'].isin(train_pitcher & test_pitcher)), 'pitcherCommon'] = np.nan
    df.loc[~(df['batterCommon'].isin(train_batter & test_batter)), 'batterCommon'] = np.nan
    df['pitcherCommon'] = create_pitcher_team_feature(df['pitcherCommon'].values, df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)
    df['batterCommon'] = create_batter_team_feature(df['batterCommon'].values, df['topTeam'].values, df['bottomTeam'].values, df['inning_top'].values)
    
    df['base_all'] = df['b1']*(10**0) + df['b2']*(10**1) + df['b3']*(10**2)

    return df

def fast_groupby_sampling_idx(df, groupby_cols, sample_size, seed=42):
    np.random.seed(seed)
    return np.concatenate(list(map(lambda x: np.random.choice(x, sample_size), list(df.groupby(groupby_cols, as_index=False).indices.values()))))


def sampling(train_df, sampling_num):
    new_train_df = pd.DataFrame()
    for i in tqdm(range(sampling_num)):
        new_train_df_sub = train_df.loc[fast_groupby_sampling_idx(train_df, groupby_cols=['gameID', 'inning', 'O'], sample_size=1, seed=i)]
        new_train_df_sub['subgameID'] = ((new_train_df_sub['gameID']*100).astype(str) + str(i).zfill(2)).astype(float)
        new_train_df = new_train_df.append(new_train_df_sub)
    return new_train_df

def create_pre_forward_group_features(df, groupby_cols, target_col):
    groupby_str = '_'.join(groupby_cols)
    df[f'{target_col}_{groupby_str}_pre1'] = df.groupby(groupby_cols)[target_col].shift(1)
    df[f'{target_col}_{groupby_str}_pre2'] = df.groupby(groupby_cols)[target_col].shift(2)
    df[f'{target_col}_{groupby_str}_forward1'] = df.groupby(groupby_cols)[target_col].shift(-1)
    df[f'{target_col}_{groupby_str}_forward2'] = df.groupby(groupby_cols)[target_col].shift(-2)
    
    if df[target_col].dtype in ['int8', 'int16', 'int32', 'int64', 'float32', 'float64']:
        df[f'{target_col}_{groupby_str}_diff_pre1'] = df[target_col] - df[f'{target_col}_{groupby_str}_pre1']
        df[f'{target_col}_{groupby_str}_diff_pre2'] = df[target_col] - df[f'{target_col}_{groupby_str}_pre2']
        df[f'{target_col}_{groupby_str}_diff_pre3'] = df[f'{target_col}_{groupby_str}_pre1'] - df[f'{target_col}_{groupby_str}_pre2']

        df[f'{target_col}_{groupby_str}_diff_forward1'] = df[target_col] - df[f'{target_col}_{groupby_str}_forward1']
        df[f'{target_col}_{groupby_str}_diff_forward2'] = df[target_col] - df[f'{target_col}_{groupby_str}_forward2']
        df[f'{target_col}_{groupby_str}_diff_forward3'] = df[f'{target_col}_{groupby_str}_forward1'] - df[f'{target_col}_{groupby_str}_forward2']
    
    else:
        df[f'{target_col}_{groupby_str}_diff_pre1'] = df[[target_col, f'{target_col}_{groupby_str}_pre1']].astype(str).apply(lambda x: 1 if x[target_col] < x[f'{target_col}_{groupby_str}_pre1'] else 0, axis=1)
        df[f'{target_col}_{groupby_str}_diff_pre2'] = df[[target_col, f'{target_col}_{groupby_str}_pre2']].astype(str).apply(lambda x: 1 if x[target_col] < x[f'{target_col}_{groupby_str}_pre2'] else 0, axis=1)
        df[f'{target_col}_{groupby_str}_diff_pre3'] = df[[f'{target_col}_{groupby_str}_pre1', f'{target_col}_{groupby_str}_pre2']].astype(str).apply(lambda x: 1 if x[f'{target_col}_{groupby_str}_pre1'] < x[f'{target_col}_{groupby_str}_pre2'] else 0, axis=1)

        df[f'{target_col}_{groupby_str}_diff_forward1'] = df[[target_col, f'{target_col}_{groupby_str}_forward1']].astype(str).apply(lambda x: 1 if x[target_col] < x[f'{target_col}_{groupby_str}_forward1'] else 0, axis=1)
        df[f'{target_col}_{groupby_str}_diff_forward2'] = df[[target_col, f'{target_col}_{groupby_str}_forward2']].astype(str).apply(lambda x: 1 if x[target_col] < x[f'{target_col}_{groupby_str}_forward2'] else 0, axis=1)
        df[f'{target_col}_{groupby_str}_diff_forward3'] = df[[f'{target_col}_{groupby_str}_forward1', f'{target_col}_{groupby_str}_forward2']].astype(str).apply(lambda x: 1 if x[f'{target_col}_{groupby_str}_forward1'] < x[f'{target_col}_{groupby_str}_forward2'] else 0, axis=1)

    return df


def has_9thbottom(df):
    df_g = df.groupby(['subgameID'])['inning'].unique().map(lambda x: 1 if '9回裏' in x else 0).reset_index().rename(columns={'inning': 'has_9thbottom'})
    df = df.merge(df_g, on=['subgameID'], how='left')
    return df

def has_out(df):
    df_g = df.groupby(['subgameID', 'inning'])['O'].unique().map(lambda x: 1 if 2 in x else 0).reset_index().rename(columns={'O': 'has_out2'})
    df = df.merge(df_g, on=['subgameID', 'inning'], how='left')
    df_g = df.groupby(['subgameID', 'inning'])['O'].unique().map(lambda x: 1 if 1 in x else 0).reset_index().rename(columns={'O': 'has_out1'})
    df = df.merge(df_g, on=['subgameID', 'inning'], how='left')
    return df

def create_pitching_order(df):
    df_g = df.groupby(['subgameID', 'pitcherTeam'])['pitcher'].unique().explode().reset_index()
    df_g['pitching_order_in_subgameID'] = df_g.groupby(['subgameID', 'pitcherTeam']).cumcount() + 1
    df = df.merge(df_g, on=['subgameID', 'pitcherTeam', 'pitcher'], how='left')
    return df

def create_batting_order(df):
    df_g = df.groupby(['subgameID', 'batterTeam'])['batter'].unique().explode().reset_index()
    df_g['batting_order_in_subgameID'] = df_g.groupby(['subgameID', 'batterTeam']).cumcount() + 1
    df = df.merge(df_g, on=['subgameID', 'batterTeam', 'batter'], how='left')
    return df

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, NMF, TruncatedSVD

def create_batter_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'batterTeam'])['batter'].agg(list).reset_index()
    df_g['batter'] = df_g['batter'].map(lambda x: ' '.join(x))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['batter'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)
    
    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'batter_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['batter'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'batterTeam'], how='left')

    return df

def create_pitcher_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'pitcherTeam'])['pitcher'].agg(list).reset_index()
    df_g['pitcher'] = df_g['pitcher'].map(lambda x: ' '.join(x))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['pitcher'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)

    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'pitcher_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['pitcher'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'pitcherTeam'], how='left')

    return df

def create_batting_order_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'batterTeam'])['batting_order_in_subgameID'].agg(list).reset_index()
    df_g['batting_order_in_subgameID'] = df_g['batting_order_in_subgameID'].map(lambda x: ' '.join(map('order{}'.format, x)))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['batting_order_in_subgameID'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)
    
    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'batting_order_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['batting_order_in_subgameID'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'batterTeam'], how='left')

    return df

def create_pitching_order_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'pitcherTeam'])['pitching_order_in_subgameID'].agg(list).reset_index()
    df_g['pitching_order_in_subgameID'] = df_g['pitching_order_in_subgameID'].map(lambda x: ' '.join(map('order{}'.format, x)))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['pitching_order_in_subgameID'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)
    
    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'pitching_order_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['pitching_order_in_subgameID'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'pitcherTeam'], how='left')

    return df

def create_batting_order_inning_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'inning_num'])['batting_order_in_subgameID'].agg(list).reset_index()
    df_g['batting_order_in_subgameID'] = df_g['batting_order_in_subgameID'].map(lambda x: ' '.join(map('order{}'.format, x)))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['batting_order_in_subgameID'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)
    
    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'batting_order_inning_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['batting_order_in_subgameID'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'inning_num'], how='left')

    return df

def create_pitching_order_inning_tfidf(df, n_components=5, compressors=['pca']):
    df_g = df.groupby(['subgameID', 'inning_num'])['pitching_order_in_subgameID'].agg(list).reset_index()
    df_g['pitching_order_in_subgameID'] = df_g['pitching_order_in_subgameID'].map(lambda x: ' '.join(map('order{}'.format, x)))
    vectorizer = TfidfVectorizer()
    input_x = vectorizer.fit_transform(df_g['pitching_order_in_subgameID'].values)
    input_x = pd.DataFrame(input_x.toarray())
    mms = MinMaxScaler()
    input_x = mms.fit_transform(input_x)
    
    for c in compressors:
        if c == 'pca':
            compressor = PCA(n_components=n_components, random_state=42)
        elif c == 'nmf':
            compressor = NMF(n_components=n_components, random_state=42)
        elif c == 'svd':
            compressor = TruncatedSVD(n_components=n_components, random_state=42)
        compressed = compressor.fit_transform(input_x)
        compressed_df = pd.DataFrame(compressed, columns=[f'pitching_order_inning_tfidf_{c}_{n}' for n in range(n_components)])
        df_g_compressed = pd.concat([df_g, compressed_df], axis=1)
        df_g_compressed.drop(columns=['pitching_order_in_subgameID'], inplace=True)
        df = df.merge(df_g_compressed, on=['subgameID', 'inning_num'], how='left')

    return df

In [5]:
df, sample_submission = read_data(INPUT_DIR)

def create_features(df, sampling_num=5):
    df = create_base_features(df)
    df = fillna_pitcherHand(df)
    df = batter_isPitcher(df)
    df = fillna_batterHand(df)
    
    ### Sampling
    train_df = df[df['test']==0].reset_index(drop=True)
    test_df = df[df['test']==1].reset_index(drop=True)
    train_df = sampling(train_df, sampling_num)
    test_df['subgameID'] = (test_df['gameID'] * 100).astype(float)
    df = pd.concat([train_df, test_df]).reset_index(drop=True)

    ### After Sampling
    df = create_pre_forward_group_features(df, groupby_cols=['subgameID', 'inning_num'], target_col='base_all')
    df = has_9thbottom(df)
    df = has_out(df)

    df = create_pitching_order(df)
    df = create_batting_order(df)
    df = create_pre_forward_group_features(df, groupby_cols=['subgameID', 'batterCommon'], target_col='pitcher')
    df = create_pre_forward_group_features(df, ['subgameID', 'batterTeam'], target_col='pitching_order_in_subgameID')
    df = create_pre_forward_group_features(df, ['subgameID', 'batterTeam'], target_col='batting_order_in_subgameID')
    
    df = create_pitcher_tfidf(df, n_components=30, compressors=['nmf'])
    df = create_pitching_order_tfidf(df, n_components=10, compressors=['nmf'])
    df = create_batting_order_tfidf(df, n_components=10, compressors=['nmf'])
    
    df = create_pitching_order_inning_tfidf(df, n_components=3, compressors=['pca'])
    df = create_batting_order_inning_tfidf(df, n_components=3, compressors=['pca'])

    df['out_cumsum_BS'] = df['BS'] + df['out_cumsum']*(10**2)
    df['out_cumsum_BSO'] = df['BS'] + df['out_cumsum']*(10**3)
    df['out_cumsum_base_all'] = df['base_all'] + df['out_cumsum']*(10**3)
    df = create_pre_forward_group_features(df, groupby_cols=['subgameID', 'inning_num'], target_col='out_cumsum_BS')
    return df


df = create_features(df, sampling_num=CFG.sampling_num)
df.info()

train shape = (20400, 24)
test shape = (33808, 14)
sample_submission shape = (33808, 2)


  0%|          | 0/10 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63728 entries, 0 to 63727
Columns: 160 entries, id to out_cumsum_BS_subgameID_inning_num_diff_forward3
dtypes: bool(3), datetime64[ns](1), float64(107), int64(25), object(24)
memory usage: 77.0+ MB


In [6]:
cat_feats = [c for c in df.columns if df[c].dtype in ['object', 'bool']]

drop_feats = [
'id',
'gameID',
'inning', 
'subgameID',
'pitchType',
'speed',
'ballPositionLabel',
'ballX',
'ballY',
'dir',
'dist',
'battingType',
'isOuts',
'y', 
'test',
'startDayTime',
'startTime',
'pitcher',
'batter',
'bgTop',
'bgBottom',
'place',
'batterHand',
'totalPitchingCount',
]

In [7]:
from sklearn.preprocessing import LabelEncoder

def label_encoding(df, cat_feats):
    labelenc_instances = {}
    df[cat_feats] = df[cat_feats].fillna('nan')
    for c in cat_feats:
        lbl = LabelEncoder()
        df[c] = lbl.fit_transform(df[c].astype(str))
        labelenc_instances[c] = lbl
    return df, labelenc_instances

df, labelenc_instances = label_encoding(df, cat_feats)
print (labelenc_instances.keys())

dict_keys(['b1', 'b2', 'b3', 'pitcher', 'batter', 'batterHand', 'inning', 'pitchType', 'speed', 'ballPositionLabel', 'ballY', 'dir', 'battingType', 'isOuts', 'startTime', 'bottomTeam', 'topTeam', 'date', 'pitcherTeam', 'batterTeam', 'pitcherCommon', 'batterCommon', 'pitcherHand', 'pitcher_subgameID_batterCommon_pre1', 'pitcher_subgameID_batterCommon_pre2', 'pitcher_subgameID_batterCommon_forward1', 'pitcher_subgameID_batterCommon_forward2'])


In [8]:
def agg(df, agg_cols):
    old_cols = list(df.columns)
    for c in tqdm(agg_cols):
        new_feature = '{}_{}_{}'.format('_'.join(c['groupby']), c['agg'], c['target'])
        
        if c['agg'] == 'mean_diff':
            df[new_feature] = df.groupby(c['groupby'])[c['target']].transform('mean') - df[c['target']]
        elif c['agg'] == 'mean_ratio':
            df[new_feature] = df.groupby(c['groupby'])[c['target']].transform('mean') / (1+df[c['target']])
        elif c['agg'] == 'median_diff':
            df[new_feature] = df.groupby(c['groupby'])[c['target']].transform('median') - df[c['target']]
        elif c['agg'] == 'median_ratio':
            df[new_feature] = df.groupby(c['groupby'])[c['target']].transform('median') / (1+df[c['target']])
        else:
            df[new_feature] = df.groupby(c['groupby'])[c['target']].transform(c['agg'])

    new_cols = list(set(list(df.columns)) - set(old_cols))
    return df, new_cols

def create_agg_feature(df, groupby_cols, target_cols, aggs):
    agg_cols = []
    for g in groupby_cols:
        for t in target_cols:
            for a in aggs:
                agg_d = {}
                agg_d['groupby'] = g
                agg_d['target'] = t
                agg_d['agg'] = a
                agg_cols.append(agg_d)

    df, new_cols = agg(df, agg_cols)

    return df, new_cols

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

groupby_cols = [
    ['subgameID', 'pitcherCommon'],
    ['subgameID', 'batterCommon'],
    ['subgameID', 'pitcherHand', 'batterHand'],
]
target_cols = [
    'b1', 
    'b2', 
    'b3', 
    'totalPitchingCount',
]
aggs = [
    'mean',
    'std',
    'skew',
    'median',
    'mean_diff',
    'mean_ratio',
]
df, new_cols = create_agg_feature(df, groupby_cols, target_cols, aggs)

input_x = df[new_cols].fillna(0)
mms = MinMaxScaler()
input_x = mms.fit_transform(input_x)
n_components = 20
pca = PCA(n_components=n_components, random_state=42)
transformed = pca.fit_transform(input_x)
pca_df = pd.DataFrame(transformed, columns=[f'pca1_{n}' for n in range(n_components)])
df = pd.concat([df, pca_df], axis=1)

from scipy.stats import ks_2samp
diff_feats = []
for c in new_cols:
    d1 = df[df['test']==0][c].values
    d2 = df[df['test']==1][c].values
    s = ks_2samp(d1, d2).statistic
    if s > 0.03:
        diff_feats.append(c)

for c in diff_feats:
    if not c in drop_feats:
        drop_feats.append(c)

        
        
groupby_cols = [
    ['batterCommon',],
]
target_cols = [
    'batting_order_in_subgameID'
]
aggs = [
    'mean',
    'std',
    'skew',
    'median',
    'mean_diff',
    'mean_ratio',
    'median_diff',
]
df, new_cols = create_agg_feature(df, groupby_cols, target_cols, aggs)


input_x = df[new_cols].fillna(0)
mms = MinMaxScaler()
input_x = mms.fit_transform(input_x)
n_components = 3
pca = PCA(n_components=n_components, random_state=42)
transformed = pca.fit_transform(input_x)
pca_df = pd.DataFrame(transformed, columns=[f'pca2_{n}' for n in range(n_components)])
df = pd.concat([df, pca_df], axis=1)

df.info()

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63728 entries, 0 to 63727
Columns: 262 entries, id to pca2_2
dtypes: datetime64[ns](1), float64(209), int64(52)
memory usage: 127.9 MB


In [10]:
train_df = df[df['test']==0].reset_index(drop=True)
test_df = df[df['test']==1].reset_index(drop=True)
print ('train_df.shape={}, test_df.shape={}'.format(train_df.shape, test_df.shape))

train_df.shape=(29920, 262), test_df.shape=(33808, 262)


In [11]:
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

class RandomGroupKFold:
    def __init__(self, n_splits=4, shuffle=True, random_state=42):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X=None, y=None, groups=None):
        kf = KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)
        unique_ids = groups.unique()
        for tr_group_idx, va_group_idx in kf.split(unique_ids):
            # split group
            tr_group, va_group = unique_ids[tr_group_idx], unique_ids[va_group_idx]
            train_idx = np.where(groups.isin(tr_group))[0]
            val_idx = np.where(groups.isin(va_group))[0]
            yield train_idx, val_idx


def create_folds(df, how_split, seeds, n_splits, target_col, group_col):
    for seed in seeds:
        df[f'fold_{seed}'] = 9999
        if how_split == 'KFold':
            kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
            for fold, (_, valid_idx) in enumerate(kf.split(df)):
                df.loc[df.iloc[valid_idx].index, f'fold_{seed}'] = fold
                
        elif how_split == 'RandomGroupKFold':
            kf = RandomGroupKFold(n_splits=n_splits, random_state=seed)
            for fold, (_, valid_idx) in enumerate(kf.split(df, df[target_col], df[group_col])):
                df.loc[df.iloc[valid_idx].index, f'fold_{seed}'] = fold
             
    return df



train_df = create_folds(
    df=train_df,
    how_split=CFG.how_split,
    seeds=CFG.seeds, 
    n_splits=CFG.n_splits, 
    target_col=CFG.target, 
    group_col=CFG.group_col
)
train_df

,id,totalPitchingCount,B,S,O,b1,b2,b3,pitcher,batter,batterHand,gameID,inning,pitchType,speed,ballPositionLabel,ballX,ballY,dir,dist,battingType,isOuts,y,test,startTime,bottomTeam,bgBottom,topTeam,place,startDayTime,bgTop,date,BS,BSO,inning_num,inning_top,inning_num_half,out_cumsum,pitcherTeam,batterTeam,pitcherCommon,batterCommon,base_all,pitcherHand,batter_isPitcher,subgameID,base_all_subgameID_inning_num_pre1,base_all_subgameID_inning_num_pre2,base_all_subgameID_inning_num_forward1,base_all_subgameID_inning_num_forward2,base_all_subgameID_inning_num_diff_pre1,base_all_subgameID_inning_num_diff_pre2,base_all_subgameID_inning_num_diff_pre3,base_all_subgameID_inning_num_diff_forward1,base_all_subgameID_inning_num_diff_forward2,base_all_subgameID_inning_num_diff_forward3,has_9thbottom,has_out2,has_out1,pitching_order_in_subgameID,batting_order_in_subgameID,pitcher_subgameID_batterCommon_pre1,pitcher_subgameID_batterCommon_pre2,pitcher_subgameID_batterCommon_forward1,pitcher_subgameID_batterCommon_forward2,pitcher_subgameID_batterCommon_diff_pre1,pitcher_subgameID_batterCommon_diff_pre2,pitcher_subgameID_batterCommon_diff_pre3,pitcher_subgameID_batterCommon_diff_forward1,pitcher_subgameID_batterCommon_diff_forward2,pitcher_subgameID_batterCommon_diff_forward3,pitching_order_in_subgameID_subgameID_batterTeam_pre1,pitching_order_in_subgameID_subgameID_batterTeam_pre2,pitching_order_in_subgameID_subgameID_batterTeam_forward1,pitching_order_in_subgameID_subgameID_batterTeam_forward2,pitching_order_in_subgameID_subgameID_batterTeam_diff_pre1,pitching_order_in_subgameID_subgameID_batterTeam_diff_pre2,pitching_order_in_subgameID_subgameID_batterTeam_diff_pre3,pitching_order_in_subgameID_subgameID_batterTeam_diff_forward1,pitching_order_in_subgameID_subgameID_batterTeam_diff_forward2,pitching_order_in_subgameID_subgameID_batterTeam_diff_forward3,batting_order_in_subgameID_subgameID_batterTeam_pre1,batting_order_in_subgameID_subgameID_batterTeam_pre2,batting_order_in_subgameID_subgameID_batterTeam_forward1,batting_order_in_subgameID_subgameID_batterTeam_forward2,batting_order_in_subgameID_subgameID_batterTeam_diff_pre1,batting_order_in_subgameID_subgameID_batterTeam_diff_pre2,batting_order_in_subgameID_subgameID_batterTeam_diff_pre3,batting_order_in_subgameID_subgameID_batterTeam_diff_forward1,batting_order_in_subgameID_subgameID_batterTeam_diff_forward2,batting_order_in_subgameID_subgameID_batterTeam_diff_forward3,pitcher_tfidf_nmf_0,pitcher_tfidf_nmf_1,pitcher_tfidf_nmf_2,pitcher_tfidf_nmf_3,pitcher_tfidf_nmf_4,pitcher_tfidf_nmf_5,pitcher_tfidf_nmf_6,pitcher_tfidf_nmf_7,pitcher_tfidf_nmf_8,pitcher_tfidf_nmf_9,pitcher_tfidf_nmf_10,pitcher_tfidf_nmf_11,pitcher_tfidf_nmf_12,pitcher_tfidf_nmf_13,pitcher_tfidf_nmf_14,pitcher_tfidf_nmf_15,pitcher_tfidf_nmf_16,pitcher_tfidf_nmf_17,pitcher_tfidf_nmf_18,pitcher_tfidf_nmf_19,pitcher_tfidf_nmf_20,pitcher_tfidf_nmf_21,pitcher_tfidf_nmf_22,pitcher_tfidf_nmf_23,pitcher_tfidf_nmf_24,pitcher_tfidf_nmf_25,pitcher_tfidf_nmf_26,pitcher_tfidf_nmf_27,pitcher_tfidf_nmf_28,pitcher_tfidf_nmf_29,pitching_order_tfidf_nmf_0,pitching_order_tfidf_nmf_1,pitching_order_tfidf_nmf_2,pitching_order_tfidf_nmf_3,pitching_order_tfidf_nmf_4,pitching_order_tfidf_nmf_5,pitching_order_tfidf_nmf_6,pitching_order_tfidf_nmf_7,pitching_order_tfidf_nmf_8,pitching_order_tfidf_nmf_9,batting_order_tfidf_nmf_0,batting_order_tfidf_nmf_1,batting_order_tfidf_nmf_2,batting_order_tfidf_nmf_3,batting_order_tfidf_nmf_4,batting_order_tfidf_nmf_5,batting_order_tfidf_nmf_6,batting_order_tfidf_nmf_7,batting_order_tfidf_nmf_8,batting_order_tfidf_nmf_9,pitching_order_inning_tfidf_pca_0,pitching_order_inning_tfidf_pca_1,pitching_order_inning_tfidf_pca_2,batting_order_inning_tfidf_pca_0,batting_order_inning_tfidf_pca_1,batting_order_inning_tfidf_pca_2,out_cumsum_BS,out_cumsum_BSO,out_cumsum_base_all,out_cumsum_BS_subgameID_inning_num_pre1,out_cumsum_BS_subgameID_inning_num_pre2,out_cumsum_BS_subgameID_inning_num_forward1,out_cumsum_BS_subgameID_inni

In [12]:
oof_preds_set = []
test_preds_set = []
feature_importance = pd.DataFrame()

In [13]:
##############################################
# LightGBM
##############################################

import lightgbm as lgb
import logging
from sklearn.metrics import f1_score

def get_score(y_true, y_pred):
    score= {}
    score['f1'] = round(f1_score(y_true, y_pred, average='macro'), 5)
    return score

def feval_f1(y_true, y_pred):
    y_pred = np.argmax(y_pred.reshape(CFG.num_class,-1), axis=0)
    return 'f1_macro', f1_score(y_true, y_pred, average='macro'), True


train_feats = [f for f in df.columns if f not in drop_feats]

oof_preds = np.zeros((len(train_df), CFG.num_class)).astype(np.float32)
test_preds = np.zeros((len(test_df), CFG.num_class)).astype(np.float32)

for seed in tqdm(CFG.seeds):
    for fold in range(CFG.n_splits):
        train_idx = train_df[train_df[f'fold_{seed}']!=fold].index
        valid_idx = train_df[train_df[f'fold_{seed}']==fold].index

        train_x, train_y = train_df.loc[train_idx], train_df.loc[train_idx][CFG.target]
        valid_x, valid_y = train_df.loc[valid_idx], train_df.loc[valid_idx][CFG.target]
        test_x = test_df.copy()

        train_x = train_x[train_feats]
        valid_x = valid_x[train_feats]
        test_x = test_x[train_feats]

        print(f'train_x.shape = {train_x.shape}, train_y.shape = {train_y.shape}')
        print(f'valid_x.shape = {valid_x.shape}, valid_y.shape = {valid_y.shape}')

        params = {
            "objective" : "multiclass", 
            "num_class": CFG.num_class,
            "boosting" : "gbdt",
            "metric" : "None", 
            'class_weight': 'balanced',
            'max_bin': 128,
            'num_leaves': 48,
            'feature_fraction': 0.8,
            'learning_rate': 0.05,
            "seed": seed,
            "verbosity": -1
        }

        # ------- Start Training
        model = lgb.LGBMClassifier(**params)
        model.fit(
            train_x, train_y,
            eval_set=(valid_x, valid_y),
            eval_metric=feval_f1,
            verbose=False,
            early_stopping_rounds=100,
        )
        best_iter = model.best_iteration_

        # validation prediction
        preds = model.predict_proba(valid_x, num_iteration=best_iter)
        oof_preds[valid_idx] += preds / len(CFG.seeds)
        fold_score = get_score(valid_y, np.argmax(preds, axis=1))
        print(f'Fold={fold} fold_score = {fold_score}')

        # test prediction
        preds = model.predict_proba(test_x, num_iteration=best_iter)
        test_preds[:] += preds / (len(CFG.seeds) * CFG.n_splits)

oof_preds_set.append(oof_preds)
test_preds_set.append(test_preds)

oof_score = get_score(train_df[CFG.target].values, np.argmax(oof_preds, axis=1))
print(f'LGB seed={seed} oof_score = {oof_score}')

  0%|          | 0/5 [00:00<?, ?it/s]

train_x.shape = (23730, 188), train_y.shape = (23730,)
valid_x.shape = (6190, 188), valid_y.shape = (6190,)
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Fold=0 fold_score = {'f1': 0.18114}
train_x.shape = (23890, 188), train_y.shape = (23890,)
valid_x.shape = (6030, 188), valid_y.shape = (6030,)
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Fold=1 fold_score = {'f1': 0.1752}
train_x.shape = (23850, 188), train_y.shape = (23850,)
valid_x.shape = (6070, 188), valid_y.shape = (6070,)
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warni

In [14]:
len(oof_preds_set), len(test_preds_set)

(1, 1)

In [15]:
oof_preds = np.mean(oof_preds_set, axis=0)
test_preds = np.mean(test_preds_set, axis=0)

In [16]:
print ('============ mean oof_preds_set ============')
y_true = train_df[CFG.target].values
y_pred = np.argmax(oof_preds, axis=1)
oof_score = get_score(y_true, y_pred)
print('mean oof_score = {}'.format(oof_score))

============ mean oof_preds_set ============
mean oof_score = {'f1': 0.18581}


In [19]:
##############################################
# SUBMISSION
##############################################

sample_submission = test_df[['id']].copy()
sample_submission['y'] = 0
test_df['preds'] = np.argmax(test_preds, axis=1).copy()
sample_submission[CFG.target] = test_df[['id', 'preds']].sort_values('id')['preds'].values
oof_score = round(oof_score['f1'], 6)
subm_path = f'./submission_{oof_score}.csv'
sample_submission.to_csv(subm_path, index=False)
print ('subm file created: {}'.format(subm_path))

subm file created: ./submission_0.18581.csv
